In [1]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET
import random
import shutil
import yaml
import cv2
import numpy as np
import os
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
model = YOLO("yolo11x.pt") # 원하는 크기 모델 입력(n, s, m, l, x 등 원하는 모델을 로드함)
print(type(model.names),len(model.names))

<class 'dict'> 80


In [3]:
curr_dir = os.getcwd()
test_loc = os.path.join(curr_dir,"test")
results = model.predict(source = test_loc,save=True)


image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 14 cars, 8 trucks, 111.4ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 1 person, 7 cars, 1 truck, 1 handbag, 107.6ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 5 persons, 1 car, 1 motorcycle, 2 backpacks, 1 handbag, 1 cell phone, 101.6ms
Speed: 0.7ms preprocess, 106.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict


In [4]:
model_save_path = os.path.join(curr_dir, "my_test_model.pt")  # 모델 저장 경로
detailed_model = YOLO(model_save_path)

In [8]:
# 현재 디렉토리와 테스트 경로 설정
curr_dir = os.getcwd()
test_loc = os.path.join(curr_dir, "test")
crop_save_dir = os.path.join(curr_dir, "crops")

# 모델 로드
model = YOLO("yolo11x.pt")  # 첫 번째 모델 (기본 차량 감지용)
detailed_model = YOLO("my_test_model.pt")  # 두 번째 모델 (세분화 모델)

# 첫 번째 모델로 감지 수행
results = model.predict(source=test_loc, save=True)

# 감지된 결과에서 'car'만 추출
for result in results:
    img = result.orig_img  # 원본 이미지
    detections = result.boxes  # 감지된 바운딩 박스들
    
    for i, box in enumerate(detections):
        cls = int(box.cls)  # 클래스 ID
        if result.names[cls] == "car":  # 'car' 클래스만 필터링
            print(result.path, result.names[cls] )
            # 바운딩 박스 좌표
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            buffer_w = int((x2 - x1) * 0.1)  # 너비의 10%
            buffer_h = int((y2 - y1) * 0.1)  # 높이의 10%
            
            x1 = max(0, x1 - buffer_w)
            y1 = max(0, y1 - buffer_h)
            x2 = min(img.shape[1], x2 + buffer_w)
            y2 = min(img.shape[0], y2 + buffer_h)


            # 'car' 영역을 잘라냄
            car_crop = img[y1:y2, x1:x2]
            crop_filename = os.path.join(crop_save_dir, f"car_crop_{i:03}.jpg")
            cv2.imwrite(crop_filename, car_crop)    
            print(crop_filename)
            detailed_results = detailed_model.predict(source=crop_filename, conf=0.5)
            
            for detailed_result in detailed_results:
                if len(detailed_result) == 0:
                    print(f">>>>>>>>>>>>>>>>>>>>>No object {crop_filename}")
                else:
                    print(f">>>>>>>>>>>>>>>>>>>>>Yes object {crop_filename}")
                    for detailed_box in detailed_result.boxes:
                        detailed_cls = int(detailed_box.cls)
                        detailed_label = detailed_result.names[detailed_cls]
                        print(f"==============Detected vehicle {crop_filename} //  {detailed_label} in")



image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 14 cars, 8 trucks, 22.9ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 1 person, 7 cars, 1 truck, 1 handbag, 18.4ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 5 persons, 1 car, 1 motorcycle, 2 backpacks, 1 handbag, 1 cell phone, 16.0ms
Speed: 1.6ms preprocess, 19.1ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict35
/app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png car
/app/yolo_structure/crops/car_crop_000.jpg

image 1/1 /app/yolo_structure/crops/car_crop_000.jpg: 352x416 (no detections), 18.4ms
Speed: 2.3ms preprocess, 18.4ms inference, 4.5ms postprocess per image at shape (1, 3, 352, 416)
>>>>>>>>>>>>>>>>>>>>>No object /app/yolo_structure/crops/car_crop_000.jpg
/app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png ca

In [64]:
crop_filename

'/app/yolo_structure/crops/car_crop_005.jpg'